In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

#Need py38 environment for this to run

import sys
print("Python version")
print (sys.version)

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot
print(kale.__version__)

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

# ---- Create initial population

pop = holo.population.Pop_Illustris()

# ---- Apply population modifiers
redz = cosmo.a_to_z(pop.scafa)

hards = [
    holo.evolution.Hard_GW,
    holo.evolution.Sesana_Scattering(),
    holo.evolution.Dynamical_Friction_NFW(attenuate=False),
]

f_edd = 0.1
all_evols = {}

all_eccen = [0.0, 0.2, 0.8]
for eccen in all_eccen:
    acc_instance = holo.accretion.Accretion(accmod='Siwek22', f_edd = f_edd, eccen=eccen, subpc=True)
    evo = holo.evolution.Evolution(pop, hards, nsteps = 100, debug=True, acc=acc_instance)
    evo.evolve()
    all_evols['evol_e=%.2f' %eccen] = evo

In [ ]:
for eccen in all_eccen:
    evo = all_evols['evol_e=%.2f' %eccen]
    all_evols['q_init_e=%.2f' %eccen] = []
    all_evols['q_final_e=%.2f' %eccen] = []
    masses = evo.mass
    for i in np.arange(0,np.shape(masses)[0]):
        inds_t_pos = evo.tlook[i] > 0
        masses = (evo.mass[i][inds_t_pos])
        if len(evo.tlook[i][inds_t_pos]) > 5:
            if masses.T[0][0] > masses.T[1][0]:
                primary_init = masses.T[0][0]
                secondary_init = masses.T[1][0]
            else:
                primary_init = masses.T[1][0]
                secondary_init = masses.T[0][0]
            if masses.T[0][-1] > masses.T[1][-1]:
                primary_final = masses.T[0][-1]
                secondary_final = masses.T[1][-1]
            else:
                primary_final = masses.T[1][-1]
                secondary_final = masses.T[0][-1]
            q_init = secondary_init/primary_init
            q_final = secondary_final/primary_final
            if q_init>=0.1:
                all_evols['q_init_e=%.2f' %eccen].append(q_init)
                all_evols['q_final_e=%.2f' %eccen].append(q_final)
        

In [ ]:
import random
import seaborn as sns
palette = sns.color_palette('colorblind')
all_colors = {
'e=0.00': palette[0],
'e=0.20': palette[1],
'e=0.40': palette[2],
'e=0.60': palette[3],
'e=0.80': palette[4],
}

fs = 20
nbins = 100
bins = np.linspace(1.e-1, 1.0, nbins)
logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))

fig, axs = plt.subplots(len(all_eccen)+1,1,figsize=(15,20))
fig.suptitle('Mass Ratio evolution of MBHBs', fontsize=fs)
for ax,eccen in zip(axs,all_eccen):
    ax.hist(all_evols['q_init_e=%.2f' %eccen], bins = logbins, log=True, \
            label=r'$q_{\rm init},$' + ' mean=%.2f' %(np.mean(all_evols['q_init_e=%.2f' %eccen])), \
            alpha=0.7, color='gray')
    ax.hist(all_evols['q_final_e=%.2f' %eccen], bins = logbins, log=True, \
            label=r'$q_{\rm final},$' + ' mean=%.2f' %(np.mean(all_evols['q_final_e=%.2f' %eccen])),\
            alpha=0.7, color=all_colors['e=%.2f' %eccen])
    ax.legend(fontsize=fs)
    ax.set_title(r'$e_{\rm b} = $' + '%.2f, %d binaries.' %(eccen,len(all_evols['q_final_e=%.2f' %eccen])), fontsize=fs)
    ax.set_xscale('log')
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)

for eccen in all_eccen:
    ax = axs[-1]
    ax.hist(all_evols['q_final_e=%.2f' %eccen], bins = logbins, log=True, \
            label=r'$q_{\rm final},$' + ' mean=%.2f' %(np.mean(all_evols['q_final_e=%.2f' %eccen])),\
            alpha=0.7, color=all_colors['e=%.2f' %eccen])
    ax.legend(fontsize=fs)
    ax.set_title(r'$e_{\rm b} = $' + '%.2f' %(eccen), fontsize=fs)
    ax.set_xscale('log')
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)

    plt.tight_layout()